<a href="https://colab.research.google.com/github/andilMc/Chatterbox_colab_mc/blob/main/Chatterbox_TTS_Colab_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Setup and Installation (Robust Version)
# Clone repository and install dependencies with error handling

# Remove existing directory and clear cache
!rm -rf Chatterbox-TTS-Server
!pip cache purge

# Clone repository
!git clone https://github.com/devnen/Chatterbox-TTS-Server.git
%cd Chatterbox-TTS-Server

print("✅ Repository cloned. Installing core dependencies...")

# Install PyTorch with compatible torchvision
!pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 --index-url https://download.pytorch.org/whl/cu121 -q

# Install your Colab-compatible chatterbox fork
!pip install git+https://github.com/devnen/chatterbox.git -q

print("✅ Core TTS components installed. Installing server dependencies...")

# Install essential server requirements (skip problematic packages)
!pip install fastapi uvicorn pyyaml soundfile librosa safetensors -q
!pip install python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm -q
!pip install pydub audiotsm -q

# Try to install parselmouth (may fail on some systems)
!pip install parselmouth -q || echo "Parselmouth installation failed - unvoiced segment removal will be disabled"

print("✅ Installation complete! Some optional packages may have been skipped.")

Files removed: 0
Cloning into 'Chatterbox-TTS-Server'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 302 (delta 135), reused 116 (delta 116), pack-reused 142 (from 1)
Receiving objects: 100% (302/302), 18.94 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (148/148), done.
/content/Chatterbox-TTS-Server
✅ Repository cloned. Installing core dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/6

In [ ]:
# @title 2. Launch the Server! ⚡
# Starts the TTS server with memory optimization

import threading
import time
import uvicorn
import gc
from google.colab.output import serve_kernel_port_as_window

port = 8004

print("🛑 Stopping existing servers...")
!fuser -k 8004/tcp 2>/dev/null || echo "Port clear"
!pkill -f "uvicorn.*server" 2>/dev/null || echo "No running servers"

print("🧹 Clearing memory...")
gc.collect()

%cd /content/Chatterbox-TTS-Server

def run_server():
    try:
        from server import app
        print("✅ Server starting...")
        # Add memory-efficient settings
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=port,
            log_level="warning",  # Reduce log verbosity
            access_log=False      # Disable access logs to save memory
        )
    except Exception as e:
        print(f"❌ Server error: {e}")

print("🚀 Starting TTS server...")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

time.sleep(25)  # Extra time for model loading

print("🎉 Server ready! Click below:")
serve_kernel_port_as_window(port)

print("\n💡 Tips to prevent crashes:")
print("• Use shorter texts (under 500 characters)")
print("• Wait for each generation to complete before starting another")
print("• If it crashes, just re-run this cell")